In [6]:
import numpy as np
import functools
from tests import * # імпортуємо тести для функцій

## Еталони

In [7]:
zero = np.array([[1, 1, 1],
                 [1, 0, 1],
                 [1, 0, 1],
                 [1, 0, 1],
                 [1, 1, 1]])

one  = np.array([[0, 1, 0],
                 [0, 1, 0],
                 [0, 1, 0],
                 [0, 1, 0],
                 [0, 1, 0]])

two  = np.array([[1, 1, 1],
                 [0, 0, 1],
                 [1, 1, 1],
                 [1, 0, 0],
                 [1, 1, 1]])

three =np.array([[1, 1, 1],
                 [0, 0, 1],
                 [1, 1, 1],
                 [0, 0, 1],
                 [1, 1, 1]])

four = np.array([[1, 0, 1],
                 [1, 0, 1],
                 [1, 1, 1],
                 [0, 0, 1],
                 [0, 0, 1]])

five = np.array([[1, 1, 1],
                 [1, 0, 0],
                 [1, 1, 1],
                 [0, 0, 1],
                 [1, 1, 1]])

six =  np.array([[1, 1, 1],
                 [1, 0, 0],
                 [1, 1, 1],
                 [1, 0, 1],
                 [1, 1, 1]])

seven =np.array([[1, 1, 1],
                 [0, 0, 1],
                 [0, 1, 0],
                 [1, 0, 0],
                 [1, 0, 0]])

eight =np.array([[1, 1, 1],
                 [1, 0, 1],
                 [1, 1, 1],
                 [1, 0, 1],
                 [1, 1, 1]])


nine = np.array([[1, 1, 1],
                 [1, 0, 1],
                 [1, 1, 1],
                 [0, 0, 1],
                 [0, 0, 1]])

In [8]:
Y1 = [zero, one, two, three, four, five, six, seven, eight, nine] # еталонні зображення
Y=[]

In [9]:
sc1 = 20    # наскільки збільшуємо висоту
sc2 = 33   # ... ширину
for i in range(len(Y1)):
    Y.append(upscale(Y1[i], sc1,sc2))

## Генеруємо гістограми

In [12]:
histogram = set_histogram(np.random.randint(0,100))
histogram

array([0.2       , 0.072     , 0.016     , 0.00533333, 0.008     ,
       0.17866667, 0.20266667, 0.128     , 0.05866667, 0.13066667])

## Генеруємо вхідне зображення
### повертаємо зобр. і суму цифр

In [13]:
def task(Y, t, p):# генеруємо таск довжиною t з ймовірністю p
    r = np.random.choice(10, t, p=histogram)
    x_etalone = Y[r[0]]
    for i in range(1,t):
        x_etalone = np.concatenate((x_etalone,Y[r[i]]),axis=1)
    n = Y[0].shape[0]
    m = Y[0].shape[1]
    print(sum(r))
    ksi = np.random.binomial(size=n*m*t, n=1, p=p).reshape((n,m*t))
    f = (x_etalone^ksi, sum(r)) 
    return f # повертаємо таск і суму чисел (ті що випали)

## Рахуємо ймовірності

In [14]:
def recog(X_arr,p):
    probab= []                      # остаточні ймовірності
    for ks in range(10):           # зовнішня сума по к штрих
        sum_ki = np.empty(10)       # для суми в знаменнику
        for ki in range(10):           # для суми в знаменнику
            stepin = np.sum((Y[ki]^X_arr) - (Y[ks]^X_arr))    # степінь в знаменнику для p/1-p
            #print(stepin)
            qwerty = np.power(p/(1-p), stepin)               # кусок
            #print((Y[ki]^X_arr).sum() , (Y[ks]^X_arr).sum())
            sum_ki[ki] = histogram[ki]*qwerty                # додаємо в суму 
            #print(stepin,sum_ki[ki] )
        znam = np.sum(sum_ki)                             # рахуємо знаменник остаточно
        #print(znam)
        r = histogram[ks]/znam                            # чисельник/знаменник
        probab.append(r)                             # додаємо ймовірність
    return probab                                         # повертаємо масив

## Рекурсивно рахуємо ймов-ті для під-зображень

In [15]:
t = 20                   # кількість цифр
p = 0.3                  # рівень шуму 
v = task(Y, t,p)[0]       # генеруємо таск
X = np.hsplit(v, t)     #розбиваємо таск по цифрах 
#recog(X[0],p)        

86


In [16]:
@functools.lru_cache(None)        # та ж функція що була раніше але з новими підрахунками
def f(t):
    if t==1:
        return recog(X[t-1],p)    # умова виходу з рекурсії
    else:
        res = []                  # масив в якому будуть остаточні ймовірності
        for d in range(0,9*t+1):  
            k2 = []
            for k in range (0,10):
                if k>=d-9*(t-1) and k<=d:          
                    ert = recog(X[t-1], p)   # p(Kt|Xt)
                    k2.append( f(t-1)[d-k]*ert[k]) # рахуємо рекурсивно 
            res.append(np.sum(k2)) 
      #print(np.sum(k2))
    return res

In [17]:
a = f(t)

#print(a)   # виводимо всі ймовірності
print(np.argmax(a)) # сума яку вгадуємо 
f.cache_clear() # для очистки (для кожного нового таску свій препроцесинг)

C:\Users\max\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: overflow encountered in power
  


86


## Перевірка подільності на `3`

In [18]:
print(checking(a,3))

False
